In [1]:
import pandas as pd

In [12]:
csv = r"D:\系统默认\Documents\gradProject\rawData\haiKouWeather_clean.csv"
df = pd.read_csv(csv,encoding='GB18030')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   日期      184 non-null    object 
 1   星期      184 non-null    object 
 2   最高温度    184 non-null    float64
 3   最低温度    184 non-null    float64
 4   天气      184 non-null    object 
 5   风向      184 non-null    object 
 6   风力      184 non-null    float64
dtypes: float64(3), object(4)
memory usage: 10.2+ KB


In [28]:
df2 = df.drop('风向',axis=1)
df2

,日期,星期,最高温度,最低温度,天气,风力
0,2017-05-01,星期一,32.0,25.0,多云,5.0
1,2017-05-02,星期二,34.0,25.0,多云,2.0
2,2017-05-03,星期三,36.0,26.0,多云,5.0
3,2017-05-04,星期四,32.0,25.0,大到暴雨,4.0
4,2017-05-05,星期五,30.0,24.0,雷阵雨,2.0
...,...,...,...,...,...,...
179,2017-10-27,星期五,21.0,0.0,多云,3.5
180,2017-10-28,星期六,21.0,0.0,多云,3.5
181,2017-10-29,星期日,22.0,0.0,多云,4.5
182,2017-10-30,星期一,26.0,22.0,多云,3.0


### 更改列名 中->英

In [29]:
columns_rename={'日期':'date','星期':'week','最高温度':'temp_h','最低温度':'temp_l','天气':'weather','风力':'wind'}
df2.rename(columns=columns_rename,inplace=True)

In [30]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   date     184 non-null    object 
 1   week     184 non-null    object 
 2   temp_h   184 non-null    float64
 3   temp_l   184 non-null    float64
 4   weather  184 non-null    object 
 5   wind     184 non-null    float64
dtypes: float64(3), object(3)
memory usage: 8.8+ KB


### Week Mapping

In [32]:
df2['week'][1]

' 星期二 '

In [33]:
week_map={' 星期一 ':1,' 星期二 ':2,' 星期三 ':3,' 星期四 ':4,' 星期五 ':5,' 星期六 ':6,' 星期日 ':7}
df2['week']=df2['week'].map(week_map)
df2

,date,week,temp_h,temp_l,weather,wind
0,2017-05-01,1,32.0,25.0,多云,5.0
1,2017-05-02,2,34.0,25.0,多云,2.0
2,2017-05-03,3,36.0,26.0,多云,5.0
3,2017-05-04,4,32.0,25.0,大到暴雨,4.0
4,2017-05-05,5,30.0,24.0,雷阵雨,2.0
...,...,...,...,...,...,...
179,2017-10-27,5,21.0,0.0,多云,3.5
180,2017-10-28,6,21.0,0.0,多云,3.5
181,2017-10-29,7,22.0,0.0,多云,4.5
182,2017-10-30,1,26.0,22.0,多云,3.0


### 将最低温度为0的异常数据替换为当月最低温度的平均值

In [38]:
df2['date'] = pd.to_datetime(df2['date'])

In [56]:
df2.groupby(df2['date'].dt.month)['temp_l'].mean()

date
5     24.806452
6     26.500000
7     25.645161
8     26.161290
9     25.800000
10    20.419355
Name: temp_l, dtype: float64

In [59]:
df2['temp_l'] = df2['temp_l'].replace(0.0, 20.4)
df2

,date,week,temp_h,temp_l,weather,wind
0,2017-05-01,1,32.0,25.0,多云,5.0
1,2017-05-02,2,34.0,25.0,多云,2.0
2,2017-05-03,3,36.0,26.0,多云,5.0
3,2017-05-04,4,32.0,25.0,大到暴雨,4.0
4,2017-05-05,5,30.0,24.0,雷阵雨,2.0
...,...,...,...,...,...,...
179,2017-10-27,5,21.0,20.4,多云,3.5
180,2017-10-28,6,21.0,20.4,多云,3.5
181,2017-10-29,7,22.0,20.4,多云,4.5
182,2017-10-30,1,26.0,22.0,多云,3.0


In [61]:
df2['temp_l'].min()

19.0

### Weather Mapping

In [63]:
weather_counts = df2.groupby('weather')['weather'].agg('count')
weather_counts

weather
中到大雨     4
中雨       4
多云      69
大到暴雨     1
大暴雨      1
大雨       6
小到中雨     1
暴雨       1
阵雨       9
雷阵雨     88
Name: weather, dtype: int64

In [64]:
len(weather_counts)

10

In [65]:
weather_map={'多云':1,'阵雨':2,'雷阵雨':2,'小到中雨':3,'中雨':3,'中到大雨':4,'大雨':4,'大到暴雨':5,'暴雨':5,'大暴雨':5}
df2['weather']=df2['weather'].map(weather_map)
df2

,date,week,temp_h,temp_l,weather,wind
0,2017-05-01,1,32.0,25.0,1,5.0
1,2017-05-02,2,34.0,25.0,1,2.0
2,2017-05-03,3,36.0,26.0,1,5.0
3,2017-05-04,4,32.0,25.0,5,4.0
4,2017-05-05,5,30.0,24.0,2,2.0
...,...,...,...,...,...,...
179,2017-10-27,5,21.0,20.4,1,3.5
180,2017-10-28,6,21.0,20.4,1,3.5
181,2017-10-29,7,22.0,20.4,1,4.5
182,2017-10-30,1,26.0,22.0,1,3.0


In [66]:
weather_counts = df2.groupby('weather')['weather'].agg('count')
weather_counts

weather
1    69
2    97
3     5
4    10
5     3
Name: weather, dtype: int64

### 添加一列：是否为工作日

In [70]:
df2['workday'] = df2['week'].map(lambda x:'1' if x<6 else '0')
df2

,date,week,temp_h,temp_l,weather,wind,workday
0,2017-05-01,1,32.0,25.0,1,5.0,1
1,2017-05-02,2,34.0,25.0,1,2.0,1
2,2017-05-03,3,36.0,26.0,1,5.0,1
3,2017-05-04,4,32.0,25.0,5,4.0,1
4,2017-05-05,5,30.0,24.0,2,2.0,1
...,...,...,...,...,...,...,...
179,2017-10-27,5,21.0,20.4,1,3.5,1
180,2017-10-28,6,21.0,20.4,1,3.5,0
181,2017-10-29,7,22.0,20.4,1,4.5,0
182,2017-10-30,1,26.0,22.0,1,3.0,1


### 导出

In [71]:
csv_save = r'D:\系统默认\Documents\gradProject\rawData\data1\weather.csv'
df2.to_csv(csv_save,index=False)